In [1]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf 
import tensorflow_datasets as tfds
keras = tf.keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 43733 files belonging to 6 classes.


2022-08-03 05:53:09.032714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 05:53:09.047145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 05:53:09.049778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 05:53:09.052896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 349639 files belonging to 6 classes.


In [6]:
base_model = keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(6, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_ds, validation_data=validation_ds, epochs=3)

Epoch 1/3


2022-08-03 05:53:35.551872: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


10927/10927 [==============================] - 364s 33ms/step - loss: 1.3610 - accuracy: 0.4582 - val_loss: 1.2880 - val_accuracy: 0.4903
Epoch 2/3
10927/10927 [==============================] - 358s 33ms/step - loss: 1.2842 - accuracy: 0.4893 - val_loss: 1.2788 - val_accuracy: 0.4925
Epoch 3/3
10927/10927 [==============================] - 360s 33ms/step - loss: 1.2754 - accuracy: 0.4930 - val_loss: 1.2812 - val_accuracy: 0.4901


In [11]:
for layer in base_model.layers:
    layer.trainable = True

In [12]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [13]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_ds, validation_data=validation_ds, epochs=20, callbacks=[es])

Epoch 1/20
10927/10927 [==============================] - 727s 66ms/step - loss: 0.9077 - accuracy: 0.6431 - val_loss: 0.7828 - val_accuracy: 0.6928
Epoch 2/20
10927/10927 [==============================] - 722s 66ms/step - loss: 0.7202 - accuracy: 0.7182 - val_loss: 0.7053 - val_accuracy: 0.7293
Epoch 3/20
10927/10927 [==============================] - 724s 66ms/step - loss: 0.6366 - accuracy: 0.7538 - val_loss: 0.6931 - val_accuracy: 0.7382
Epoch 4/20
10927/10927 [==============================] - 726s 66ms/step - loss: 0.5744 - accuracy: 0.7788 - val_loss: 0.6475 - val_accuracy: 0.7586
Epoch 5/20
10927/10927 [==============================] - 721s 66ms/step - loss: 0.5235 - accuracy: 0.8003 - val_loss: 0.6389 - val_accuracy: 0.7652
Epoch 6/20
10927/10927 [==============================] - 722s 66ms/step - loss: 0.4810 - accuracy: 0.8172 - val_loss: 0.6450 - val_accuracy: 0.7652
Epoch 7/20
10927/10927 [==============================] - 722s 66ms/step - loss: 0.4439 - accuracy: 0.8318